# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')
#openai.api_key = ""


#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.  
**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

In [10]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note:** This and all other lab notebooks of this course use OpenAI library version `0.28.0`. 

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the `get_completion` function:

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [12]:
text = f"""
How do connection types differ? Connection types determine how your users are sourced and how they authenticate. It\\’s important to understand the features of each connection type in order to select the most appropriate one for your use case. One important distinction is who is responsible for the identity lifecycle of your users. The identity lifecycle is the process of creating and maintaining your users\\' digital identity. This is a continuous process that includes onboarding, assigning resources, authentication, and updating and maintaining credentials and profile attributes. The identity lifecycle ends when a user's digital identity is permanently deleted. The entity responsible for the identity lifecycle is the Identity Provider (IdP). You may choose to manage the identity lifecycle of your users by using Auth0 as your IdP. However, in some use cases, it may make sense to leverage an existing single sign-on session from an external IdP.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence that is no more than 100 characters in length.
```{text}```
"""
response = get_completion(prompt)
print(response)

Connection types affect user sourcing, authentication, and identity lifecycle management by IdPs.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [13]:
prompt = f"""
Generate a list of three made-up Auth0 user profiles that consists of the user's name, nickname, given name, family name, and picture URL. \\
Provide them in JSON format with the following keys: name, nickname, given_name, family_name, picture.
"""
response = get_completion(prompt)
print(response)

Here is a list of three made-up Auth0 user profiles in JSON format:

```json
[
    {
        "name": "Alice Johnson",
        "nickname": "Ally",
        "given_name": "Alice",
        "family_name": "Johnson",
        "picture": "https://example.com/pictures/alice_johnson.jpg"
    },
    {
        "name": "Michael Smith",
        "nickname": "Mike",
        "given_name": "Michael",
        "family_name": "Smith",
        "picture": "https://example.com/pictures/michael_smith.jpg"
    },
    {
        "name": "Sophia Brown",
        "nickname": "Sophie",
        "given_name": "Sophia",
        "family_name": "Brown",
        "picture": "https://example.com/pictures/sophia_brown.jpg"
    }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

#### Tactic 4: "Few-shot" prompting

In [15]:
prompt = f"""
Your task is to create an assessment item that is aligned to a learning objective using a consistent style.
Learning objective: Understand the options available in Auth0 password policies such as password complexity and password history.
Assessment item:
What does password complexity refer to?
A. The number of characters required
B. The combination of character types required (CORRECT)
C. The number of authentication methods required
D. The combination of authentication methods required
Learning objective: Compare and contrast bulk migration and automatic migration.

"""
response = get_completion(prompt)
print(response)

Assessment item: 

What is a key difference between bulk migration and automatic migration in the context of user data transfer?

A. Bulk migration requires manual intervention for each user, while automatic migration processes users without manual input. (CORRECT)  
B. Automatic migration is only applicable to small user bases, while bulk migration can handle any size.  
C. Bulk migration is faster than automatic migration in all scenarios.  
D. Automatic migration is only used for password resets, while bulk migration is for complete user data transfer.  


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [16]:
learning_objective = f"""
Compare and contrast automatic migration and bulk migration
"""

course_text = f"""
Migrate Users to Auth0 with Database Connections
Migration vs. sourcing from an external database

You may already have a source of users before implementing Auth0. In an Auth0 database connection, 
you can connect your own custom database to your tenant. This solution makes you responsible for:

• Keeping the database online and publicly available
• Implementing the appropriate action scripts to manage the user lifecycle

Migrating your users to Auth0's user store frees you of these responsibilities and improves 
the performance of your login flow. Auth0 offers two distinct strategies for migrating your 
users from your existing database to Auth0's user store: bulk migration and automatic migration.

Bulk migration

The bulk migration strategy allows users to be migrated into Auth0 all at once. The bulk migration 
service supports importing passwords hashed with a number of standard hashing algorithms including bcrypt. 
The imported hash value is used only the first time the migrated user authenticates through Auth0. 
Once that happens, the user's password is hashed with Auth0's default algorithm, bcrypt.

Bulk migration can be performed using one of two methods:
- the Auth0 Management API
- the User Import/Export Users Extension

Both solutions require a specifically formatted JSON schema representing the users you would like to import. 
The table below represents the differences between the two methods:

Auth0 Management API:
- Invoke the users-imports endpoint, passing the JSON file of user profiles.
- Most flexible; you can implement a machine-to-machine application with custom logic.

Import/Export Users Extension:
- Install the extension to your tenant from the Extensions page of your Auth0 dashboard.
- Use the extension's UI to upload the JSON file of user profiles.
- Least flexible; limited to functionality of the extension.

Automatic migration

Automatic migration (also called lazy or trickle migration) migrates users individually as they sign in. 
This requires keeping both the Auth0 user store and your legacy database alive during the migration period.

When a user logs in through Auth0:
- If the user does not exist in Auth0, credentials are checked against the external database.
- If valid, username and hashed password are stored in Auth0 for future logins.

This involves:
- Creating a custom database connection in Auth0
- Enabling "Import Users to Auth0" in the connection's Settings tab
- Implementing two action scripts: Login and Get User

The Login script:
- Verifies the user in the legacy database when not found in Auth0

The Get User script:
- Executes during signup, password change, or when email is entered using Identifier First + Biometrics

If an action script succeeds, Auth0 hashes the password using bcrypt and stores the user.

Should I use my existing database or migrate?

The choice depends on your business needs. You might:
- Keep your legacy database
- Use one of the migration strategies
- Combine both approaches

Scenarios

External custom database:
- You are forced to keep your own user store

Bulk Migration:
- You need time to migrate users
- You want to explore Auth0 before fully committing

Automatic Migration:
- You want gradual migration as users log in

Caveats

External custom database:
- Must implement action scripts
- Maintain your own user store
- Make your store publicly accessible (e.g., reverse proxy)

Combining automatic and bulk migration

To reduce caveats:
- Start with automatic migration
- Later apply bulk migration for remaining users

This hybrid approach allows:
- Transparent migration for active users
- A phased, non-urgent migration process

Caveats:
- Takes longer than bulk migration alone
- Requires public access to your user store during automatic migration

Wrap up

We discussed user migration to Auth0 as an alternative to maintaining your own external custom database. 
We covered bulk and automatic migration strategies, scenarios, caveats, and a hybrid approach.
"""
# example 1
prompt_1 = f"""
Your task is to perform the following actions:
1.  Create a multiple choice assessment item that aligns with the ```{learning_objective}``` and can be answered with the information provided in ```{course_text}```. 
2. Generate one correct answer and three distractors.
3. Provide a quote from ```{course_text}``` that supports the correct answer.

"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
### Assessment Item

**Question:** Which of the following statements accurately describes the differences between bulk migration and automatic migration in the context of migrating users to Auth0?

A) Bulk migration allows users to be migrated all at once, while automatic migration migrates users individually as they sign in.

B) Automatic migration requires the legacy database to be taken offline during the migration process, while bulk migration does not.

C) Both bulk and automatic migration strategies require the legacy database to be publicly accessible at all times.

D) Bulk migration is less flexible than automatic migration because it does not require any action scripts to be implemented.

**Correct Answer:** A) Bulk migration allows users to be migrated all at once, while automatic migration migrates users individually as they sign in.

**Supporting Quote:** "The bulk migration strategy allows users to be migrated into Auth0 all at once... Automatic mi

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [11]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \\
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \\
me a flat $100k per year, and an additional $10 / square \\
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is mostly correct, but there is a small error in the calculation of the maintenance cost. Let's break it down step by step:

1. **Land cost**: The cost of land is $100 per square foot, so for \( x \) square feet, the cost is:
   \[
   100x
   \]

2. **Solar panel cost**: The cost of solar panels is $250 per square foot, so for \( x \) square feet, the cost is:
   \[
   250x
   \]

3. **Maintenance cost**: The maintenance cost consists of a flat fee of $100,000 plus an additional $10 per square foot. Therefore, the total maintenance cost for \( x \) square feet is:
   \[
   100,000 + 10x
   \]

Now, we can combine all these costs to find the total cost for the first year of operations:

\[
\text{Total cost} = \text{Land cost} + \text{Solar panel cost} + \text{Maintenance cost}
\]
\[
\text{Total cost} = 100x + 250x + (100,000 + 10x)
\]
\[
\text{Total cost} = 100x + 250x + 100,000 + 10x
\]
\[
\text{Total cost} = (100x + 250x + 10x) + 100,000
\]
\[
\text{Total cost} 

#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

## Try experimenting on your own!

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.